In [34]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler

from datetime import datetime

# Models

In [32]:
from sklearn.ensemble import IsolationForest, RandomForestClassifier

## BaseModel

In [19]:
from abc import ABC, abstractmethod
from sklearn.base import BaseEstimator

class AnomalyDetector(BaseEstimator, ABC):
    def __init__(self):
        self.model = None
        self.threshold = None

    @abstractmethod
    def fit(self, X, y=None):
        """Обучение модели"""
        pass

    @abstractmethod
    def predict(self, X):
        """Предсказание аномальности"""
        pass

    @abstractmethod
    def predict_proba(self, X):
        """Вероятность аномальности"""
        pass

## DataFrequencyDetector

In [20]:
class DataFrequencyDetector(AnomalyDetector):
    def __init__(self, window_size=3600, contamination=0.1):
        super().__init__()
        self.model = IsolationForest(contamination=contamination, random_state=42)
        self.scaler = StandardScaler()
        self.is_fitted = False

    def _prepare_features(self, X):
        """Подготовка признаков для анализа частоты обращений к данным"""
        features = np.column_stack([
            X['avg_http_requests_per_day'],
            X['avg_emails_sent_per_day'],
            X['avg_emails_received_per_day'],
            X['avg_file_copies_per_day']
        ])
        if not self.is_fitted:
            self.is_fitted = True
            return self.scaler.fit_transform(features)
        return self.scaler.transform(features)

    def fit(self, X, y=None):
        features = self._prepare_features(X)
        self.model.fit(features)
        return self

    def predict(self, X):
        features = self._prepare_features(X)
        return self.model.predict(features)

    def predict_proba(self, X):
        features = self._prepare_features(X)
        scores = -self.model.score_samples(features)
        return scores / np.max(scores) if len(scores) > 0 else scores

## DataVolumeDetector

In [21]:
class DataVolumeDetector(AnomalyDetector):
    def __init__(self, window_size=3600, contamination=0.1):
        super().__init__()
        self.model = IsolationForest(contamination=contamination, random_state=42)
        self.scaler = StandardScaler()
        self.is_fitted = False

    def _prepare_features(self, X):
        """Подготовка признаков для анализа объема данных"""
        features = np.column_stack([
            X['unique_domains'],
            X['unique_email_contacts'],
            X['unique_file_types'],
            X['unique_pcs']
        ])
        if not self.is_fitted:
            self.is_fitted = True
            return self.scaler.fit_transform(features)
        return self.scaler.transform(features)

    def fit(self, X, y=None):
        features = self._prepare_features(X)
        self.model.fit(features)
        return self

    def predict(self, X):
        features = self._prepare_features(X)
        return self.model.predict(features)

    def predict_proba(self, X):
        features = self._prepare_features(X)
        scores = -self.model.score_samples(features)
        return scores / np.max(scores) if len(scores) > 0 else scores

## FileActivityDetector

In [22]:
class FileActivityDetector(AnomalyDetector):
    def __init__(self, contamination=0.1):
        """
        Parameters:
        -----------
        contamination : float, default=0.1
            Ожидаемая доля аномалий в данных
        """
        super().__init__()
        self.model = IsolationForest(
            contamination=contamination,
            random_state=42,
            n_estimators=100
        )

    def extract_features(self, X):
        """
        Извлечение признаков для анализа файловых операций

        Parameters:
        -----------
        X : pandas.DataFrame
            Датафрейм с признаками пользователей

        Returns:
        --------
        numpy.ndarray
            Матрица признаков для анализа файловых операций
        """
        features = []

        # Основные признаки
        if 'avg_file_copies_per_day' in X.columns:
            features.append(X['avg_file_copies_per_day'].values.reshape(-1, 1))

        if 'unique_file_types' in X.columns:
            features.append(X['unique_file_types'].values.reshape(-1, 1))

        # Комбинированные признаки с другими активностями
        if 'after_hours_logon_ratio' in X.columns and 'avg_file_copies_per_day' in X.columns:
            features.append((X['after_hours_logon_ratio'] * X['avg_file_copies_per_day']).values.reshape(-1, 1))

        if 'weekend_logon_ratio' in X.columns and 'avg_file_copies_per_day' in X.columns:
            features.append((X['weekend_logon_ratio'] * X['avg_file_copies_per_day']).values.reshape(-1, 1))

        if 'device_usage_ratio' in X.columns and 'avg_file_copies_per_day' in X.columns:
            features.append((X['device_usage_ratio'] * X['avg_file_copies_per_day']).values.reshape(-1, 1))

        # Если нет признаков файловой активности, возвращаем нулевую матрицу
        if not features:
            return np.zeros((len(X), 1))

        return np.hstack(features)

    def fit(self, X, y=None):
        """
        Обучение модели

        Parameters:
        -----------
        X : pandas.DataFrame
            Датафрейм с признаками пользователей
        y : array-like, default=None
            Игнорируется, добавлен для совместимости

        Returns:
        --------
        self : object
            Возвращает себя
        """
        features = self.extract_features(X)
        self.model.fit(features)
        return self

    def predict(self, X):
        """
        Предсказание аномальности

        Parameters:
        -----------
        X : pandas.DataFrame
            Датафрейм с признаками пользователей

        Returns:
        --------
        numpy.ndarray
            Массив меток: 1 - нормальное поведение, -1 - аномальное
        """
        features = self.extract_features(X)
        return self.model.predict(features)

    def predict_proba(self, X):
        """
        Вероятность аномальности

        Parameters:
        -----------
        X : pandas.DataFrame
            Датафрейм с признаками пользователей

        Returns:
        --------
        numpy.ndarray
            Массив вероятностей аномальности
        """
        features = self.extract_features(X)
        # Преобразуем decision_function в вероятности
        scores = self.model.decision_function(features)
        # Нормализуем scores в диапазон [0, 1], где 1 - наиболее аномальное
        probs = 1 - (scores - scores.min()) / (scores.max() - scores.min())
        return probs

    def _prepare_features(self, X):
        """Подготовка признаков для анализа файловых операций"""
        features = np.column_stack([
            X['avg_file_copies_per_day'],
            X['unique_file_types'],
            X['after_hours_logon_ratio'],
            X['weekend_logon_ratio']
        ])
        return features

## GeoLocationDetector

In [6]:
!pip install geoip2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 6.9 MB/s eta 0:00:00


In [24]:
import geoip2.database

class GeoLocationDetector(AnomalyDetector):
    def __init__(self, geoip_db_path, known_good_ips=None, contamination=0.1):
        super().__init__()
        self.model = IsolationForest(contamination=contamination, random_state=42)
        self.geoip_reader = geoip2.database.Reader(geoip_db_path)
        self.known_good_ips = known_good_ips or set()

    def _get_location_features(self, ip):
        """Получение географических признаков по IP"""
        try:
            response = self.geoip_reader.city(ip)
            return [
                response.location.latitude,
                response.location.longitude,
                int(ip in self.known_good_ips)
            ]
        except:
            return [0, 0, 0]  # дефолтные значения при ошибке

    def _prepare_features(self, X):
        """Подготовка географических признаков"""
        features = []
        for ip in X['ip_address']:
            features.append(self._get_location_features(ip))
        return np.array(features)

    def fit(self, X, y=None):
        features = self._prepare_features(X)
        self.model.fit(features)
        return self

    def predict(self, X):
        features = self._prepare_features(X)
        return self.model.predict(features)

    def predict_proba(self, X):
        features = self._prepare_features(X)
        scores = -self.model.score_samples(features)
        return scores / np.max(scores)

    def __del__(self):
        """Закрываем reader при удалении объекта"""
        if hasattr(self, 'geoip_reader'):
            self.geoip_reader.close()

## ResourceAccessDetector

In [25]:
class ResourceAccessDetector(AnomalyDetector):
    def __init__(self, contamination=0.1):
        super().__init__()
        self.model = IsolationForest(contamination=contamination, random_state=42)
        self.scaler = StandardScaler()
        self.is_fitted = False

    def _prepare_features(self, X):
        """Подготовка признаков для анализа доступа к ресурсам"""
        features = np.column_stack([
            X['is_admin'],
            X['unique_pcs'],
            X['device_usage_ratio'],
            X['O'], X['C'], X['E'], X['A'], X['N']  # Психометрические характеристики
        ])
        if not self.is_fitted:
            self.is_fitted = True
            return self.scaler.fit_transform(features)
        return self.scaler.transform(features)

    def fit(self, X, y=None):
        features = self._prepare_features(X)
        self.model.fit(features)
        return self

    def predict(self, X):
        features = self._prepare_features(X)
        return self.model.predict(features)

    def predict_proba(self, X):
        features = self._prepare_features(X)
        scores = -self.model.score_samples(features)
        return scores / np.max(scores) if len(scores) > 0 else scores

## TimeActivityDetector

In [26]:
class TimeActivityDetector(AnomalyDetector):
    def __init__(self, contamination=0.1):
        super().__init__()
        self.model = IsolationForest(contamination=contamination, random_state=42)

    def _prepare_features(self, X):
        """Подготовка признаков для анализа временных паттернов"""
        features = np.column_stack([
            X['avg_logons_per_day'],
            X['weekend_logon_ratio'],
            X['after_hours_logon_ratio'],
            X['avg_device_usage_per_day'],
            X['device_usage_ratio']
        ])
        return features

    def fit(self, X, y=None):
        features = self._prepare_features(X)
        self.model.fit(features)
        return self

    def predict(self, X):
        features = self._prepare_features(X)
        return self.model.predict(features)

    def predict_proba(self, X):
        features = self._prepare_features(X)
        scores = -self.model.score_samples(features)
        return scores / np.max(scores) if len(scores) > 0 else scores

## EnsembleDetector

In [27]:
class EnsembleDetector(AnomalyDetector):
    def __init__(self, detectors, weights=None):
        """
        Parameters:
        -----------
        detectors : dict
            Словарь детекторов {name: detector}
        weights : dict, optional
            Веса для каждого детектора {name: weight}
        """
        super().__init__()
        self.detectors = detectors
        self.weights = weights or {name: 1.0 for name in detectors.keys()}

    def fit(self, X, y=None):
        """Обучение всех базовых детекторов"""
        print("\nОбучение базовых детекторов...")
        for name, detector in self.detectors.items():
            print(f"Обучение {name}...")
            detector.fit(X)
        return self

    def _get_base_predictions(self, X):
        """Получение предсказаний от всех базовых детекторов"""
        predictions = {}
        for name, detector in self.detectors.items():
            pred = detector.predict_proba(X)
            predictions[name] = pred * self.weights[name]
        return predictions

    def predict_proba(self, X):
        """Вероятностные оценки аномальности"""
        base_predictions = self._get_base_predictions(X)

        # Взвешенное среднее всех предсказаний
        weighted_sum = np.zeros(len(X))
        total_weight = 0

        for name, pred in base_predictions.items():
            weight = self.weights[name]
            weighted_sum += pred * weight
            total_weight += weight

        return weighted_sum / total_weight if total_weight > 0 else weighted_sum

    def predict(self, X, threshold=0.8):
        """
        Предсказание аномальности

        Parameters:
        -----------
        X : pandas.DataFrame
            Входные данные
        threshold : float, default=0.8
            Порог для определения аномалии

        Returns:
        --------
        numpy.ndarray
            Массив меток: -1 - аномалия, 1 - норма
        """
        probas = self.predict_proba(X)
        return np.where(probas > threshold, -1, 1)

    def update_weights(self, new_weights):
        """Обновление весов базовых детекторов"""
        self.weights.update(new_weights)

# DataLoader

In [33]:
import os
import glob

class DataLoader:
    def __init__(self, base_path):
        """
        Parameters:
        -----------
        base_path : str
            Путь к директории с данными
        """
        self.base_path = base_path
        self.datasets = ['r1', 'r2', 'r3.1']  # Добавляем r1 в список поддерживаемых датасетов
        self.current_dataset = None
        self.logon_data = None
        self.device_data = None
        self.http_data = None
        self.email_data = None
        self.file_data = None
        self.ldap_data = None
        self.psychometric_data = None
        self.features = None
        self.insiders_data = None

    def load_insiders_info(self):
        """Загрузка информации об инсайдерах"""
        insiders_data = pd.DataFrame({
            'dataset': ['r2', 'r3.1', 'r2'],
            'scenario': [1, 1, 2],
            'user': ['ONS0995', 'CSF0929', 'CCH0959'],
            'start': pd.to_datetime(['3/6/2010 1:41:56', '07/01/2010 01:24:58', '08/02/2010 10:34:31']),
            'end': pd.to_datetime(['3/20/2010 8:10:12', '07/16/2010 06:52:00', '09/30/2010 15:04:03']),
        })
        self.insiders_data = insiders_data
        return insiders_data

    def load_data(self, dataset='r3.1'):
        """Загрузка данных из указанного набора"""
        if dataset not in self.datasets:
            raise ValueError(f"Dataset {dataset} not supported. Available datasets: {self.datasets}")

        self.current_dataset = dataset
        dataset_path = os.path.join(self.base_path, dataset)

        # Загрузка информации об инсайдерах
        self.load_insiders_info()

        # Загрузка основных данных
        print(f"Loading data from {dataset}...")

        # Логи входа
        print("Loading logon data...")
        try:
            self.logon_data = pd.read_csv(os.path.join(dataset_path, 'logon.csv'))
            # Проверяем и преобразуем столбец с датой
            date_column = next((col for col in ['date', 'timestamp'] if col in self.logon_data.columns), None)
            if date_column:
                self.logon_data[date_column] = pd.to_datetime(self.logon_data[date_column])
                if date_column != 'date':
                    self.logon_data = self.logon_data.rename(columns={date_column: 'date'})
        except Exception as e:
            print(f"Error loading logon data: {e}")
            self.logon_data = pd.DataFrame(columns=['user', 'pc', 'date', 'activity'])

        # Данные устройств
        print("Loading device data...")
        try:
            self.device_data = pd.read_csv(os.path.join(dataset_path, 'device.csv'))
            date_column = next((col for col in ['date', 'timestamp'] if col in self.device_data.columns), None)
            if date_column:
                self.device_data[date_column] = pd.to_datetime(self.device_data[date_column])
                if date_column != 'date':
                    self.device_data = self.device_data.rename(columns={date_column: 'date'})
        except Exception as e:
            print(f"Error loading device data: {e}")
            self.device_data = pd.DataFrame(columns=['user', 'pc', 'date', 'activity'])

        # HTTP данные
        print("Loading HTTP data...")
        try:
            self.http_data = pd.read_csv(os.path.join(dataset_path, 'http.csv'))
            # Проверяем наличие столбца user или id
            if 'id' in self.http_data.columns and 'user' not in self.http_data.columns:
                self.http_data = self.http_data.rename(columns={'id': 'user'})

            # Проверяем и преобразуем столбец с датой
            date_column = next((col for col in ['date', 'timestamp'] if col in self.http_data.columns), None)
            if date_column:
                self.http_data[date_column] = pd.to_datetime(self.http_data[date_column])
                if date_column != 'date':
                    self.http_data = self.http_data.rename(columns={date_column: 'date'})

            # Если нет столбца url, но есть website
            if 'website' in self.http_data.columns and 'url' not in self.http_data.columns:
                self.http_data = self.http_data.rename(columns={'website': 'url'})

        except Exception as e:
            print(f"Error loading HTTP data: {e}")
            self.http_data = pd.DataFrame(columns=['user', 'pc', 'date', 'url'])

        # Email данные
        print("Loading email data...")
        try:
            self.email_data = pd.read_csv(os.path.join(dataset_path, 'email.csv'))
            date_column = next((col for col in ['date', 'timestamp'] if col in self.email_data.columns), None)
            if date_column:
                self.email_data[date_column] = pd.to_datetime(self.email_data[date_column])
                if date_column != 'date':
                    self.email_data = self.email_data.rename(columns={date_column: 'date'})
        except Exception as e:
            print(f"Error loading email data: {e}")
            self.email_data = pd.DataFrame(columns=['from', 'to', 'date'])

        # Файловые операции (только для r3.1)
        if dataset == 'r3.1':
            print("Loading file data...")
            try:
                self.file_data = pd.read_csv(os.path.join(dataset_path, 'file.csv'))
                # Проверяем и преобразуем столбец с датой
                date_column = 'date' if 'date' in self.file_data.columns else 'timestamp'
                self.file_data[date_column] = pd.to_datetime(self.file_data[date_column])
                if date_column != 'date':
                    self.file_data = self.file_data.rename(columns={date_column: 'date'})
            except Exception as e:
                print(f"Error loading file data: {e}")
                self.file_data = pd.DataFrame(columns=['user', 'pc', 'date', 'filename'])
        else:
            self.file_data = pd.DataFrame(columns=['user', 'pc', 'date', 'filename'])

        # Психометрические данные
        try:
            print("Loading psychometric data...")
            self.psychometric_data = pd.read_csv(os.path.join(dataset_path, 'psychometric.csv'))
        except Exception as e:
            print(f"Error loading psychometric data: {e}")
            self.psychometric_data = pd.DataFrame(columns=['user_id', 'O', 'C', 'E', 'A', 'N'])

        # LDAP данные
        print("Loading LDAP data...")
        try:
            ldap_files = glob.glob(os.path.join(dataset_path, 'LDAP', '*.csv'))
            if ldap_files:
                ldap_dfs = []
                for file in ldap_files:
                    try:
                        df = pd.read_csv(file)
                        month = os.path.basename(file).split('.')[0]
                        df['month'] = month
                        ldap_dfs.append(df)
                    except Exception as e:
                        print(f"Error loading LDAP file {file}: {e}")
                if ldap_dfs:
                    self.ldap_data = pd.concat(ldap_dfs, ignore_index=True)
                else:
                    self.ldap_data = pd.DataFrame(columns=['user_id', 'role', 'month'])
            else:
                print("No LDAP files found")
                self.ldap_data = pd.DataFrame(columns=['user_id', 'role', 'month'])
        except Exception as e:
            print(f"Error processing LDAP data: {e}")
            self.ldap_data = pd.DataFrame(columns=['user_id', 'role', 'month'])

    def is_weekend(self, date):
        """Проверка является ли день выходным"""
        return date.weekday() >= 5

    def is_after_hours(self, date):
        """Проверка является ли время нерабочим"""
        hour = date.hour
        return hour < 7 or hour > 18

    def prepare_features(self):
        """Подготовка признаков для обучения"""
        features = []

        # Получаем список всех пользователей
        users = pd.unique(self.logon_data['user'])

        for user in users:
            # Фильтруем данные пользователя
            user_logons = self.logon_data[self.logon_data['user'] == user]
            user_devices = self.device_data[self.device_data['user'] == user]
            user_http = self.http_data[self.http_data['user'] == user] if 'user' in self.http_data.columns else pd.DataFrame()
            user_emails = self.email_data[
                (self.email_data['from'] == user) |
                (self.email_data['to'].str.contains(user, na=False))
            ] if not self.email_data.empty else pd.DataFrame()

            # Базовые характеристики
            total_days = (user_logons['date'].max() - user_logons['date'].min()).days + 1
            if total_days == 0:  # Если все события в один день
                total_days = 1

            # Характеристики входов
            logon_features = {
                'avg_logons_per_day': len(user_logons) / total_days,
                'weekend_logon_ratio': user_logons[user_logons['date'].apply(self.is_weekend)].shape[0] / len(user_logons) if len(user_logons) > 0 else 0,
                'after_hours_logon_ratio': user_logons[user_logons['date'].apply(self.is_after_hours)].shape[0] / len(user_logons) if len(user_logons) > 0 else 0,
                'unique_pcs': user_logons['pc'].nunique(),
            }

            # Характеристики устройств
            device_features = {
                'avg_device_usage_per_day': len(user_devices) / total_days,
                'device_usage_ratio': len(user_devices[user_devices['activity'] == 'Connect']) / total_days if total_days > 0 else 0,
            }

            # HTTP характеристики
            http_features = {
                'avg_http_requests_per_day': len(user_http) / total_days if not user_http.empty else 0,
                'unique_domains': user_http['url'].apply(lambda x: x.split('/')[0]).nunique() if not user_http.empty and 'url' in user_http.columns else 0,
            }

            # Email характеристики
            email_features = {
                'avg_emails_sent_per_day': len(user_emails[user_emails['from'] == user]) / total_days if not user_emails.empty else 0,
                'avg_emails_received_per_day': len(user_emails[user_emails['to'].str.contains(user, na=False)]) / total_days if not user_emails.empty else 0,
                'unique_email_contacts': pd.concat([
                    user_emails['to'].str.split(';').explode(),
                    user_emails['from']
                ]).nunique() if not user_emails.empty else 0,
            }

            # Дополнительные характеристики для r3.1
            if self.current_dataset == 'r3.1' and not self.file_data.empty:
                user_files = self.file_data[self.file_data['user'] == user]
                file_features = {
                    'avg_file_copies_per_day': len(user_files) / total_days,
                    'unique_file_types': user_files['filename'].apply(lambda x: x.split('.')[-1] if '.' in x else '').nunique(),
                }
            else:
                file_features = {
                    'avg_file_copies_per_day': 0,
                    'unique_file_types': 0,
                }

            # Психометрические характеристики
            user_psycho = self.psychometric_data[self.psychometric_data['user_id'] == user]
            if not user_psycho.empty:
                psycho_features = user_psycho.iloc[0][['O', 'C', 'E', 'A', 'N']].to_dict()
            else:
                psycho_features = {
                    'O': 0, 'C': 0, 'E': 0, 'A': 0, 'N': 0
                }

            # LDAP характеристики
            user_ldap = self.ldap_data[self.ldap_data['user_id'] == user]
            ldap_features = {
                'is_admin': 1 if not user_ldap.empty and user_ldap.iloc[-1]['role'] == 'ITAdmin' else 0,
            }

            # Объединяем все характеристики
            user_features = {
                'user_id': user,
                **logon_features,
                **device_features,
                **http_features,
                **email_features,
                **file_features,
                **psycho_features,
                **ldap_features,
            }

            features.append(user_features)

        # Создаем DataFrame
        self.features = pd.DataFrame(features)

        # Нормализация числовых признаков
        numeric_columns = self.features.select_dtypes(include=[np.number]).columns
        numeric_columns = numeric_columns.drop(['user_id', 'is_admin']) if 'user_id' in numeric_columns else numeric_columns

        if not numeric_columns.empty:
            scaler = StandardScaler()
            self.features[numeric_columns] = scaler.fit_transform(self.features[numeric_columns])

        return self.features

    def get_user_timeline(self, user):
        """Получение временной линии событий пользователя"""
        timeline = []

        # Добавляем логины
        logons = self.logon_data[self.logon_data['user'] == user]
        for _, row in logons.iterrows():
            timeline.append({
                'date': row['date'],
                'type': 'logon',
                'details': f"{row['activity']} on {row['pc']}"
            })

        # Добавляем использование устройств
        devices = self.device_data[self.device_data['user'] == user]
        for _, row in devices.iterrows():
            timeline.append({
                'date': row['date'],
                'type': 'device',
                'details': f"{row['activity']} on {row['pc']}"
            })

        # Добавляем HTTP запросы
        http = self.http_data[self.http_data['user'] == user]
        for _, row in http.iterrows():
            timeline.append({
                'date': row['date'],
                'type': 'http',
                'details': f"Visited {row['url']}"
            })

        # Добавляем email активность
        emails_sent = self.email_data[self.email_data['from'] == user]
        emails_received = self.email_data[self.email_data['to'].str.contains(user, na=False)]

        for _, row in emails_sent.iterrows():
            timeline.append({
                'date': row['date'],
                'type': 'email',
                'details': f"Sent email to {row['to']}"
            })

        for _, row in emails_received.iterrows():
            timeline.append({
                'date': row['date'],
                'type': 'email',
                'details': f"Received email from {row['from']}"
            })

        # Добавляем файловые операции для r3.1
        if self.current_dataset == 'r3.1' and self.file_data is not None:
            files = self.file_data[self.file_data['user'] == user]
            for _, row in files.iterrows():
                timeline.append({
                    'date': row['date'],
                    'type': 'file',
                    'details': f"Copied file {row['filename']} on {row['pc']}"
                })

        # Сортируем по времени
        timeline = pd.DataFrame(timeline)
        timeline = timeline.sort_values('date')

        return timeline

# Main

In [35]:
from sklearn.metrics import precision_score, recall_score, f1_score
from tqdm import tqdm
import time

def create_anomaly_detection_system():
    """Создание системы детектирования аномалий"""
    print("\n[1/4] Инициализация детекторов...")

    # Создаем базовые детекторы с повышенной чувствительностью
    detectors = {
        'time_activity': TimeActivityDetector(contamination=0.05),
        'data_frequency': DataFrequencyDetector(window_size=3600, contamination=0.05),
        'data_volume': DataVolumeDetector(window_size=3600, contamination=0.05),
        'resource_access': ResourceAccessDetector(contamination=0.05),
        'file_activity': FileActivityDetector(contamination=0.05)
    }

    print("Настройка весов детекторов...")
    # Создаем веса для каждого детектора
    weights = {
        'time_activity': 1.5,    # Повышенный вес для временных паттернов
        'data_frequency': 1.2,   # Повышенный вес для частоты обращений
        'data_volume': 1.2,      # Повышенный вес для объема данных
        'resource_access': 1.5,  # Повышенный вес для доступа к ресурсам
        'file_activity': 1.5     # Повышенный вес для файловых операций
    }

    # Создаем ансамбль
    print("Создание ансамбля детекторов...")
    ensemble = EnsembleDetector(detectors, weights)
    return ensemble

def evaluate_detection(features, predictions, probabilities, loader, dataset):
    """Оценка качества обнаружения инсайдеров"""
    print("\n[3/4] Оценка результатов детектирования...")

    print("Получение списка реальных инсайдеров...")
    # Получаем список реальных инсайдеров для текущего датасета
    real_insiders = loader.insiders_data[loader.insiders_data['dataset'] == dataset]['user'].tolist()

    print("Подготовка массива истинных меток...")
    # Создаем массив истинных меток
    y_true = np.zeros(len(features))
    for idx, user in tqdm(enumerate(features['user_id']),
                         desc="Разметка пользователей",
                         total=len(features)):
        if user in real_insiders:
            y_true[idx] = 1

    print("Вычисление метрик качества...")
    # Преобразуем предсказания из [-1, 1] в [0, 1]
    y_pred = (predictions == -1).astype(int)

    # Вычисляем метрики
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)

    print("\nМетрики качества обнаружения:")
    print(f"Precision (точность): {precision:.3f}")
    print(f"Recall (полнота): {recall:.3f}")
    print(f"F1-score: {f1:.3f}")

    # Анализ результатов
    print("\nАнализ результатов обнаружения...")
    detected_insiders = []
    missed_insiders = []
    false_positives = []

    for idx, (user, prob) in tqdm(enumerate(zip(features['user_id'], probabilities)),
                                 desc="Анализ предсказаний",
                                 total=len(features)):
        if y_pred[idx] == 1:  # Если обнаружена аномалия
            if user in real_insiders:
                detected_insiders.append((user, prob))
            else:
                false_positives.append((user, prob))
        elif user in real_insiders:
            missed_insiders.append((user, prob))

    # Вывод результатов
    print("\nРезультаты анализа:")
    print(f"Всего пользователей проанализировано: {len(features)}")
    print(f"Обнаружено потенциальных аномалий: {len(detected_insiders) + len(false_positives)}")
    print(f"Правильно обнаружено инсайдеров: {len(detected_insiders)}")
    print(f"Ложных срабатываний: {len(false_positives)}")
    print(f"Пропущено инсайдеров: {len(missed_insiders)}")

    print("\nОбнаруженные инсайдеры:")
    for user, prob in detected_insiders:
        insider_info = loader.insiders_data[
            (loader.insiders_data['dataset'] == dataset) &
            (loader.insiders_data['user'] == user)
        ].iloc[0]
        print(f"- {user} (вероятность: {prob:.3f})")
        print(f"  Сценарий: {insider_info['scenario']}")
        print(f"  Период активности: {insider_info['start']} - {insider_info['end']}")

    if missed_insiders:
        print("\nПропущенные инсайдеры:")
        for user, prob in missed_insiders:
            print(f"- {user} (вероятность: {prob:.3f})")

    print("\nЛожные срабатывания (топ-5 по вероятности):")
    for user, prob in sorted(false_positives, key=lambda x: x[1], reverse=True)[:5]:
        print(f"- {user} (вероятность: {prob:.3f})")

def main():
    start_time = time.time()
    print("\n=== Запуск системы обнаружения инсайдеров ===")

    # Анализируем все доступные датасеты
    datasets = ['r1', 'r2', 'r3.1']

    for dataset in datasets:
        print(f"\n{'='*50}")
        print(f"Анализ датасета {dataset}")
        print(f"{'='*50}")

        # Загружаем данные
        print(f"\n[1/5] Загрузка данных из датасета {dataset}...")
        loader = DataLoader('dataset')
        loader.load_data(dataset)

        # Подготавливаем признаки
        print("\n[2/5] Подготовка признаков...")
        features = loader.prepare_features()
        print(f"Извлечено {features.shape[1]} признаков для {features.shape[0]} пользователей")

        # Используем только признаки без меток
        X = features.drop(['user_id'], axis=1)

        # Создаем систему детектирования
        print("\n[3/5] Создание системы детектирования...")
        system = create_anomaly_detection_system()

        # Обучаем систему
        print("\n[4/5] Обучение моделей...")
        system.fit(X)

        # Получаем предсказания для всех пользователей
        print("\n[5/5] Анализ поведения пользователей...")
        predictions = system.predict(X)
        probabilities = system.predict_proba(X)

        # Оцениваем качество обнаружения
        evaluate_detection(features, predictions, probabilities, loader, dataset)

        # Анализируем пользователей с высоким риском
        print("\nПодробный анализ пользователей высокого риска...")
        high_risk_mask = probabilities > 0.8
        high_risk_users = features.loc[high_risk_mask, 'user_id']

        print(f"\nНайдено {len(high_risk_users)} пользователей с высоким риском (p > 0.8)")

        for user in tqdm(high_risk_users, desc="Анализ пользователей высокого риска"):
            print(f"\nАнализ пользователя {user}:")
            # Получаем временную линию событий пользователя
            timeline = loader.get_user_timeline(user)
            print(f"Всего событий: {len(timeline)}")
            if not timeline.empty:
                print("Типы событий:")
                print(timeline['type'].value_counts())

                # Показываем примеры подозрительных действий
                print("\nПримеры последних действий:")
                print(timeline.tail().to_string())

                # Если это реальный инсайдер, показываем дополнительную информацию
                insider_info = loader.insiders_data[
                    (loader.insiders_data['dataset'] == dataset) &
                    (loader.insiders_data['user'] == user)
                ]
                if not insider_info.empty:
                    info = insider_info.iloc[0]
                    print("\nПОДТВЕРЖДЕННЫЙ ИНСАЙДЕР!")
                    print(f"Сценарий: {info['scenario']}")
                    print(f"Период активности: {info['start']} - {info['end']}")

    end_time = time.time()
    execution_time = end_time - start_time
    print(f"\n=== Анализ всех датасетов завершен за {execution_time:.2f} секунд ===")

if __name__ == '__main__':
    main()


=== Запуск системы обнаружения инсайдеров ===

Анализ датасета r1

[1/5] Загрузка данных из датасета r1...
Loading data from r1...
Loading logon data...
Error loading logon data: [Errno 2] No such file or directory: 'dataset/r1/logon.csv'
Loading device data...
Error loading device data: [Errno 2] No such file or directory: 'dataset/r1/device.csv'
Loading HTTP data...
Error loading HTTP data: [Errno 2] No such file or directory: 'dataset/r1/http.csv'
Loading email data...
Error loading email data: [Errno 2] No such file or directory: 'dataset/r1/email.csv'
Loading psychometric data...
Error loading psychometric data: [Errno 2] No such file or directory: 'dataset/r1/psychometric.csv'
Loading LDAP data...
No LDAP files found

[2/5] Подготовка признаков...
Извлечено 0 признаков для 0 пользователей


KeyError: "['user_id'] not found in axis"